# Ансамблинг моделей детекции дефектов сварки

## Ensemble-boxes baseline

В данном ноутбуке представлен код создания ансамбля из двух моделей object detection

In [51]:
import csv
import ensemble_boxes

## 1. Загружаем сабмиты сделанные с использованием одной модели

In [160]:
files = []

detr_data = {}
yolo_data = {}
big_data = {}

# FPN model submit from detectron2
with open("detr_submission_new.csv") as f:
    detr_reader = csv.reader(f, delimiter=";")

    for i, row in enumerate(detr_reader):
        # пропускаем заголовок файла
        if i == 0:
            continue

        if row[0] not in detr_data:
            # сохраняем координаты прямоугольника, класс и уверенность модели
            detr_data[row[0]] = {
                "bboxes": [[float(row[i]) for i in range(2, 6)]],
                "class": [int(row[1])],
                "conf": [float(row[6])],
            }
        else:
            detr_data[row[0]]["bboxes"].append([float(row[i]) for i in range(2, 6)])
            detr_data[row[0]]["class"].append(int(row[1]))
            detr_data[row[0]]["conf"].append(float(row[6]))

# YOLOv9c model submit
with open("submission_new.csv") as f:
    yolo_reader = csv.reader(f, delimiter=";")

    for i, row in enumerate(yolo_reader):
        if i == 0:
            continue

        if row[0] not in yolo_data:
            yolo_data[row[0]] = {
                "bboxes": [[float(row[i]) for i in range(2, 6)]],
                "class": [int(row[1])],
                "conf": [float(row[6])],
            }
        else:
            yolo_data[row[0]]["bboxes"].append([float(row[i]) for i in range(2, 6)])
            yolo_data[row[0]]["class"].append(int(row[1]))
            yolo_data[row[0]]["conf"].append(float(row[6]))


создаем общий список файлов датасета

In [162]:
files += [i for i in detr_data if i not in files]
files += [i for i in yolo_data if i not in files]

len(files)

1111

# 2. Соединяем предикты моделей и удаляем дубликаты через nms

функции для перевода координат прямоугольников в формат yolo и обратно

In [163]:
def x1y1x2y2_from_xywh(bbox):
    return [bbox[0] - bbox[2] / 2, bbox[1] - bbox[3] / 2, bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2]

def xywh_from_x1y1x2y2(bbox):
    return [(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2, bbox[2] - bbox[0], bbox[3] - bbox[1]]

In [164]:
lines = []

In [165]:
for file in files:
    if file in yolo_data and file in detr_data:
        boxes_list = [
            [x1y1x2y2_from_xywh(i) for i in yolo_data[file]["bboxes"]],
            [x1y1x2y2_from_xywh(i) for i in detr_data[file]["bboxes"]]
        ]

        confs_list = [
            yolo_data[file]["conf"],
            detr_data[file]["conf"]
        ]

        classes_list = [
            yolo_data[file]["class"],
            detr_data[file]["class"]
        ]
    elif file in yolo_data:
        boxes_list = [
            [x1y1x2y2_from_xywh(i) for i in yolo_data[file]["bboxes"]],
        ]

        confs_list = [
            yolo_data[file]["conf"],
        ]

        classes_list = [
            yolo_data[file]["class"],
        ]
    else:
        boxes_list = [
            [x1y1x2y2_from_xywh(i) for i in detr_data[file]["bboxes"]]
        ]

        confs_list = [
            detr_data[file]["conf"]
        ]

        classes_list = [
            detr_data[file]["class"]
        ]

    weights = None
    iou_thr = 0.5
    skip_box_thr = 0.0001
    sigma = 0.1

    # объединяем данные
    boxes, confs, classes = ensemble_boxes.soft_nms(boxes_list, confs_list, classes_list, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
    
    for i, box in enumerate(boxes):
        lines.append([file, classes[i], *xywh_from_x1y1x2y2(box)])


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

# 3. Создание итогового файла с предсказаниями

In [159]:
with open("new_submission.csv", mode="w") as f:
    csv_writer = csv.writer(f, lineterminator="\n", delimiter=";")
    csv_writer.writerow("filename;class_id;rel_x;rel_y;width;height".split(";"))
    csv_writer.writerows(lines)